In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt
import mplcyberpunk
%matplotlib inline

# plt.style.use('ggplot')
plt.style.use("cyberpunk")

import os
os.getcwd()

'/home/jovyan/work/Capstone_3/Skincare-Recommender'

In [3]:
# Import Custom Modules
from src.helpers import *

In [4]:
# Import Data
items = pd.read_csv('data/items_final.csv')
users = pd.read_csv('data/user_final.csv')
chemicals = pd.read_csv('data/chemical_effects_final.csv')

# LDA for Ingredient Features

In [5]:
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
pyLDAvis.enable_notebook()


In [6]:
# Get Ingredients
ingredients = items['ingredients']
# Vectorize Ingredients

# Count Vec
cnt_vec = CountVectorizer(strip_accents='unicode', lowercase=True, token_pattern=r'\b[a-zA-Z]{3,}\b')
doc_term_matrix = cnt_vec.fit_transform(ingredients)
# TFIDF Vec
tfidf_vec = TfidfVectorizer(**cnt_vec.get_params())
doc_term_matrix_tfidf = tfidf_vec.fit_transform(ingredients)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1817: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


In [7]:
n_comp = 5
num_words = 10
# Perform LDA
# TF ONLY
lda_term_freq = LatentDirichletAllocation(n_components=n_comp, random_state=0)
lda_term_freq.fit(doc_term_matrix)
# TFIDF
lda_term_freq_idf = LatentDirichletAllocation(n_components=n_comp, random_state=0)
lda_term_freq_idf.fit(doc_term_matrix_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [8]:
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda_term_freq, cnt_vec, num_words)

Topics found via LDA:

Topic #0:
extract sodium acid oil glycol root alcohol glyceryl palmitoyl polypeptide

Topic #1:
extract oil glycol sodium water leaf centella asiatica acid lecithin

Topic #2:
extract leaf flower peg glycol acid sodium oil stearate fruit

Topic #3:
argan oil hyaluronate stearic fruit alcohol edta dimethicone butylene glycerin

Topic #4:
extract glycol dimethicone sodium root fruit hyaluronate disodium edta butylene


In [9]:
# Visualize LDA
LDA_prepared = pyLDAvis.sklearn.prepare(lda_term_freq, doc_term_matrix_tfidf, cnt_vec)

In [10]:
# Write to File
# pyLDAvis.save_html(LDA_prepared, 'media/previews/lda.html')

# Modeling with Surprise

In [11]:
import surprise
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate 
from surprise.model_selection import train_test_split
from surprise.model_selection import RandomizedSearchCV

from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise.model_selection import cross_validate
from surprise import KNNBaseline
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

In [15]:
# Start with SVD
data = users[['user_id', 'item_name', 'star_rating']]
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(data, reader=reader)

trainset, testset = train_test_split(data, test_size=.25)

In [16]:
# Train SVD Model
svd = SVD()
svd.fit(trainset)

In [17]:
predictions = svd.test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 1.1916
MAE:  0.9266


0.9266240338002559

Predictions look decent, with testing on different test sizes, it was found nearing 0.2 test_split resulted in best RMSE

# Modeling with LightFM